## ***Binary Classification with Bank Churn Dataset - Kaggle Playground S04E01***
### ***Objective***: To predict whether a customer will continue with the account or close it (e.g.churn)

----
## ***Table of content***
----

## Placeholder

## ***1. Importing libraries***

In [1]:
# import libraries
# libraries for data manipulation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# libraries for data preprocessing & prediction
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

## ***2. Reading data & overview of the data***

In [2]:
sample_submission = pd.read_csv('sample_submission.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

Now we start basic exploration of the dataset. My general approach for tabular data would be as follow:
- Dataset shapes
- List of features & target
- Check for data types
- Check & deal with missing values
- Basic Statistical summary (both categorical & numberical values)
- Distribution of features & target
- Correlation analysis
- Features interactions
- Outlier detection
- Hypothesis Generation. From this, we decide on further data manipulation such as encoding, feature engineering

This process is iterative, most like trial & error with ultimate goal is forming a hypothesis for effective feature engineering. So please bear with me.

### 2.1 Basic shapes of dataset

In [3]:
# Data shape
print('Train shape:', train.shape)
print('Test shape:', test.shape)

Train shape: (165034, 14)
Test shape: (110023, 13)


In [4]:
# First 05 rows of train dataset
from IPython.display import display
display(train.head())

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [5]:
# First 05 rows of the test dataset
display(test.head())

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,165034,15773898,Lucchese,586,France,Female,23.0,2,0.00,2,0.0,1.0,160976.75
1,165035,15782418,Nott,683,France,Female,46.0,2,0.00,1,1.0,0.0,72549.27
2,165036,15807120,K?,656,France,Female,34.0,7,0.00,2,1.0,0.0,138882.09
3,165037,15808905,O'Donnell,681,France,Male,36.0,8,0.00,1,1.0,0.0,113931.57
4,165038,15607314,Higgins,752,Germany,Male,38.0,10,121263.62,1,1.0,0.0,139431.00


In [6]:
# List of features in the dataset
print('Features:', train.columns)

Features: Index(['id', 'CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender',
       'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')


### 2.2 Data types of features & target

In [7]:
# Data types of the features
print('Data types:', train.dtypes)

Data types: id                   int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                float64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard          float64
IsActiveMember     float64
EstimatedSalary    float64
Exited               int64
dtype: object


In [8]:
# Summary of the dataset
train.describe()

,id,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,165034.0000,1.650340e+05,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000
mean,82516.5000,1.569201e+07,656.454373,38.125888,5.020353,55478.086689,1.554455,0.753954,0.497770,112574.822734,0.211599
std,47641.3565,7.139782e+04,80.103340,8.867205,2.806159,62817.663278,0.547154,0.430707,0.499997,50292.865585,0.408443
min,0.0000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000,0.000000
25%,41258.2500,1.563314e+07,597.000000,32.000000,3.000000,0.000000,1.000000,1.000000,0.000000,74637.570000,0.000000
50%,82516.5000,1.569017e+07,659.000000,37.000000,5.000000,0.000000,2.000000,1.000000,0.000000,117948.000000,0.000000
75%,123774.7500,1.575682e+07,710.000000,42.000000,7.000000,119939.517500,2.000000,1.000000,1.000000,155152.467500,0.000000
max,165033.0000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.000000,1.000000,199992.480000,1.000000


In [9]:
train_null = train.isnull().sum()
test_null = test.isnull().sum()
print('Train Null:\n', train_null)
print('Test Null:\n', test_null)

Train Null:
 id                 0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64
Test Null:
 id                 0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
dtype: int64


In [11]:
def classify_features_with_unique(df, categorical_threshold=20):
    feature_types = []
    unique_values = []
    
    for column in df.columns:
        if df[column].dtype == 'object':
            feature_type = 'text'
            unique = ', '.join(df[column].unique())
        elif df[column].nunique() <= categorical_threshold:
            feature_type = 'categorical'
            unique = ', '.join(map(str, df[column].unique()))
        else:
            feature_type = 'continuous'
            unique = ''
        feature_types.append(feature_type)
        unique_values.append(unique)
    
    result = pd.DataFrame({
        'Feature': df.columns,
        'Type': feature_types,
        'Unique Values': unique_values
    })
    
    return result

# Usage
classification_table = classify_features_with_unique(train)
display(classification_table)


,Feature,Type,Unique Values
0,id,continuous,
1,CustomerId,continuous,
2,Surname,text,"Okwudilichukwu, Okwudiliolisa, Hsueh, Kao, Chi..."
3,CreditScore,continuous,
4,Geography,text,"France, Spain, Germany"
5,Gender,text,"Male, Female"
6,Age,continuous,
7,Tenure,categorical,"3, 1, 10, 2, 5, 4, 8, 6, 9, 7, 0"
8,Balance,continuous,
9,NumOfProducts,categorical,"2, 1, 3, 4"


***Observation***:
- We have 01 target `Exited` as categorical
- We have 02 ID types variables, which I don't believe would show much of a value: `id` & `CustomerId`
- We have 04 continuous variables with: `CreditScore`, `Age`, `Balance` & `EstimatedSalary`. However, based on my domain expertise, we should try to categorized them into different groups. That would help improve predictability & model interpretability
- We have 03 text variables: `Surname`, `Geography` & `Gender`. Again, we could encode `Geography` & `Gender`.
- No missing value. That's great